In [5]:
!pip install kagglehub

   ---------------------------------------- 0.0/51.9 kB ? eta -:--:--
   ---------------------------------------  51.2/51.9 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 51.9/51.9 kB 1.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import os

# Readers Data

Wczytanie plików wygenerowanych w Mockaroo

In [1]:
files = {
    'data_readers1.csv',
    'data_readers2.csv',
    'data_readers3.csv',
    'data_readers4.csv',
    'data_readers5.csv'
}

In [4]:
df_list = []

for f in files:
    if os.path.exists(f):
        df_temp = pd.read_csv(f)  # wczytaj CSV do tymczasowego DataFrame
        df_list.append(df_temp)
    else:
        print(f"UWAGA: Plik {f} nie istnieje w tej ścieżce.")

Połączenie danych w jedną tabele

In [5]:
df_merged = pd.concat(df_list, ignore_index=True)
print(f"Liczba wierszy po scaleniu: {len(df_merged)}")

Liczba wierszy po scaleniu: 5000


In [6]:
df_merged.to_json("readers_data.json", orient="records", force_ascii=False)

Wyświetlenie połączonych danych

In [7]:
df_readers_json = pd.read_json('readers_data.json')
display(df_readers_json.head(15).style.hide(axis='index'))

library_card_id,first_name,last_name,email,gender,birth_year
10522,Georgianne,Ciani,gciani0@ucoz.com,Female,1976
17127,Harrie,Jedrzejewski,hjedrzejewski1@oracle.com,Female,1939
14148,Robina,Voelker,rvoelker2@upenn.edu,Female,1990
10299,Artemas,Curd,acurd3@constantcontact.com,Male,1920
12119,Belia,Banbridge,bbanbridge4@privacy.gov.au,Female,1998
17860,Clara,Umbers,cumbers5@narod.ru,Female,2004
12287,Rachel,Grayshan,rgrayshan6@ning.com,Female,1971
10976,Ardith,Evenett,aevenett7@mozilla.com,Female,1949
11902,Corny,Meadway,cmeadway8@oaic.gov.au,Male,2010
13906,Nannie,Atkins,natkins9@indiegogo.com,Female,1928


# Books Data

Import danych z Kaggle

In [9]:
import kagglehub

# Download latest version
# Download latest version
path = kagglehub.dataset_download("dylanjcastillo/7k-books-with-metadata")

print("Path to dataset files:", path)

c:\Users\Jakub\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Jakub\.cache\kagglehub\datasets\dylanjcastillo\7k-books-with-metadata\versions\3


In [10]:
files = os.listdir(path)
print(files)

['books.csv']


Wyświetlenie przykładowych danych

In [11]:
csv_file = os.path.join(path, 'books.csv')
df = pd.read_csv(csv_file)
# Dodanie kolumny ID
df.insert(0, 'ID', range(1, len(df) + 1))  # ID od 1 do liczby wierszy w danych
df.head(10).style.hide(axis='index')

ID,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
1,9780002005883,0002005883,Gilead,nan,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCPgAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api,"A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world has to offer. At its heart is a tale of the sacred bonds between fathers and sons, pitch-perfect in style and story, set to dazzle critics and readers alike.",2004.000000,3.850000,247.000000,361.000000
2,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GPgAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api,"A new 'Christie for Christmas' -- a full-length novel adapted from her acclaimed play by Charles Osborne Following BLACK COFFEE and THE UNEXPECTED GUEST comes the final Agatha Christie play novelisation, bringing her superb storytelling to a new legion of fans. Clarissa, the wife of a Foreign Office diplomat, is given to daydreaming. 'Supposing I were to come down one morning and find a dead body in the library, what should I do?' she muses. Clarissa has her chance to find out when she discovers a body in the drawing-room of her house in Kent. Desperate to dispose of the body before her husband comes home with an important foreign politician, Clarissa persuades her three house guests to become accessories and accomplices. It seems that the murdered man was not unknown to certain members of the house party (but which ones?), and the search begins for the murderer and the motive, while at the same time trying to persuade a police inspector that there has been no murder at all... SPIDER'S WEB was written in 1954 specifically for Margaret Lockwood and opened first at the Theatre Royal Nottingham before moving to the Savoy Theatre in London on 14 December 1954. With THE MOUSETRAP and WI",2000.000000,3.830000,241.000000,5164.000000
3,9780006163831,0006163831,The One Tree,nan,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQawwEACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api,Volume Two of Stephen Donaldson's acclaimed second trilogy featuing the compelling anti-hero Thomas Covenant.,1982.000000,3.970000,479.000000,172.000000
4,9780006178736,0006178731,Rage of angels,nan,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2TgANz74C&printsec=frontcover&img=1&zoom=1&source=gbs_api,"A memorable, mesmerizing heroine Jennifer -- brilliant, beautiful, an attorney on the way up until the Mafia's schemes win her the hatred of an implacable enemy -- and a love more destructive than hate. A dangerous, dramatic world The Dark Arena of organized crime and flashbulb lit courtrooms where ambitious prosecutors begin their climb to political power.",1993.000000,3.930000,512.000000,29532.000000
5,9780006280897,0006280897,The Four Loves,nan,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5XsFcpG

Usunęcie niepotrzebych kolumn

In [12]:
df = df.drop(['isbn13', 'isbn10', 'thumbnail', 'description','ratings_count'], axis=1)
df.head(10).style.hide(axis='index')

ID,title,subtitle,authors,categories,published_year,average_rating,num_pages
1,Gilead,nan,Marilynne Robinson,Fiction,2004.000000,3.850000,247.000000
2,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,2000.000000,3.830000,241.000000
3,The One Tree,nan,Stephen R. Donaldson,American fiction,1982.000000,3.970000,479.000000
4,Rage of angels,nan,Sidney Sheldon,Fiction,1993.000000,3.930000,512.000000
5,The Four Loves,nan,Clive Staples Lewis,Christian life,2002.000000,4.150000,170.000000
6,The Problem of Pain,nan,Clive Staples Lewis,Christian life,2002.000000,4.090000,176.000000
7,An Autobiography,nan,Agatha Christie,"Authors, English",1977.000000,4.270000,560.000000
8,Empires of the Monsoon,A History of the Indian Ocean and Its Invaders,Richard Hall,"Africa, East",1998.000000,4.410000,608.000000
9,The Gap Into Madness,Chaos and Order,Stephen R. Donaldson,"Hyland, Morn (Fictitious character)",1994.000000,4.150000,743.000000
10,Master of the Game,nan,Sidney Sheldon,Adventure stories,1982.000000,4.110000,489.000000


Łączenie kolumny title oraz subtitle

In [13]:
df['merged_title'] = df['title'].fillna('') + ' ' + df['subtitle'].fillna('')
df['merged_title'] = df['merged_title'].str.strip()

In [14]:
df = df.drop(['title', 'subtitle'], axis=1)

In [15]:
df.head(20).style.hide(axis='index')

ID,authors,categories,published_year,average_rating,num_pages,merged_title
1,Marilynne Robinson,Fiction,2004.000000,3.850000,247.000000,Gilead
2,Charles Osborne;Agatha Christie,Detective and mystery stories,2000.000000,3.830000,241.000000,Spider's Web A Novel
3,Stephen R. Donaldson,American fiction,1982.000000,3.970000,479.000000,The One Tree
4,Sidney Sheldon,Fiction,1993.000000,3.930000,512.000000,Rage of angels
5,Clive Staples Lewis,Christian life,2002.000000,4.150000,170.000000,The Four Loves
6,Clive Staples Lewis,Christian life,2002.000000,4.090000,176.000000,The Problem of Pain
7,Agatha Christie,"Authors, English",1977.000000,4.270000,560.000000,An Autobiography
8,Richard Hall,"Africa, East",1998.000000,4.410000,608.000000,Empires of the Monsoon A History of the Indian Ocean and Its Invaders
9,Stephen R. Donaldson,"Hyland, Morn (Fictitious character)",1994.000000,4.150000,743.000000,The Gap Into Madness Chaos and Order
10,Sidney Sheldon,Adventure stories,1982.000000,4.110000,489.000000,Master of the Game


Zamiania typu danych w kolumnach published_year oraz num_pages

In [16]:
df['published_year'] = pd.to_numeric(df['published_year'].astype(str).str.replace(',', '.', regex=False), errors='coerce').astype('Int64')
df['num_pages'] = pd.to_numeric(df['num_pages'].astype(str).str.replace(',', '.', regex=False), errors='coerce').astype('Int64')
df.head(20).style.hide(axis='index')

ID,authors,categories,published_year,average_rating,num_pages,merged_title
1,Marilynne Robinson,Fiction,2004,3.850000,247,Gilead
2,Charles Osborne;Agatha Christie,Detective and mystery stories,2000,3.830000,241,Spider's Web A Novel
3,Stephen R. Donaldson,American fiction,1982,3.970000,479,The One Tree
4,Sidney Sheldon,Fiction,1993,3.930000,512,Rage of angels
5,Clive Staples Lewis,Christian life,2002,4.150000,170,The Four Loves
6,Clive Staples Lewis,Christian life,2002,4.090000,176,The Problem of Pain
7,Agatha Christie,"Authors, English",1977,4.270000,560,An Autobiography
8,Richard Hall,"Africa, East",1998,4.410000,608,Empires of the Monsoon A History of the Indian Ocean and Its Invaders
9,Stephen R. Donaldson,"Hyland, Morn (Fictitious character)",1994,4.150000,743,The Gap Into Madness Chaos and Order
10,Sidney Sheldon,Adventure stories,1982,4.110000,489,Master of the Game


Export danych do pliku csv

In [17]:
df.to_json("books_data.json", orient="records", force_ascii=False)

Wyświetlenie danych 

In [18]:
df_books_data_json = pd.read_json('books_data.json')
# Zamiast display(df_jason(head(10)))
display(df_books_data_json.head(10).style.hide(axis='index'))

ID,authors,categories,published_year,average_rating,num_pages,merged_title
1,Marilynne Robinson,Fiction,2004.000000,3.850000,247.000000,Gilead
2,Charles Osborne;Agatha Christie,Detective and mystery stories,2000.000000,3.830000,241.000000,Spider's Web A Novel
3,Stephen R. Donaldson,American fiction,1982.000000,3.970000,479.000000,The One Tree
4,Sidney Sheldon,Fiction,1993.000000,3.930000,512.000000,Rage of angels
5,Clive Staples Lewis,Christian life,2002.000000,4.150000,170.000000,The Four Loves
6,Clive Staples Lewis,Christian life,2002.000000,4.090000,176.000000,The Problem of Pain
7,Agatha Christie,"Authors, English",1977.000000,4.270000,560.000000,An Autobiography
8,Richard Hall,"Africa, East",1998.000000,4.410000,608.000000,Empires of the Monsoon A History of the Indian Ocean and Its Invaders
9,Stephen R. Donaldson,"Hyland, Morn (Fictitious character)",1994.000000,4.150000,743.000000,The Gap Into Madness Chaos and Order
10,Sidney Sheldon,Adventure stories,1982.000000,4.110000,489.000000,Master of the Game


# Borrowing Data

In [19]:
import json
import random
from datetime import datetime, timedelta

Funkcja generująca daty 

In [20]:
def generate_dates_json(filename, num_rows=30000):
    """
    Generuje plik JSON z dwoma polami dat:
    - data wypożyczenia
    - data oddania

    Daty mieszczą się w roku 2024 i zawsze data oddania >= data wypożyczenia.
    """
    start_date = datetime(2024, 1, 1)
    end_date = datetime(2024, 12, 31)

    delta = end_date - start_date
    total_days = delta.days + 1  # +1 bo liczymy także ostatni dzień

    data = []

    for _ in range(num_rows):
        borrow_offset = random.randint(0, total_days - 1)
        borrow_date = start_date + timedelta(days=borrow_offset)

        max_return_offset = min(30, (end_date - borrow_date).days)
        return_offset = random.randint(0, max_return_offset)
        return_date = borrow_date + timedelta(days=return_offset)

        data.append({
            "borrowing_date": borrow_date.strftime('%Y-%m-%d'),
            "return_date": return_date.strftime('%Y-%m-%d')
        })

    with open(filename, 'w', encoding='utf-8') as jsonfile:
        json.dump(data, jsonfile, indent=2, ensure_ascii=False)

if __name__ == "__main__":
    generate_dates_json("borrowing_dates.json", num_rows=30000)
    print("Plik borrowing_dates.json został pomyślnie wygenerowany!")


Plik borrowing_dates.json został pomyślnie wygenerowany!


Skopiowanie danych w library_card_id do 30 000 rekordów wraz z randomizacją

In [35]:
# Path to the uploaded file
file_path = 'readers_data.json'

# Load the data from the file
with open(file_path, 'r') as file:
    data = json.load(file)

# Extract only the "library_card_id" from each record, handling nested structures
library_card_ids = []
for record in data:
    if isinstance(record["library_card_id"], dict):
        library_card_ids.append(record["library_card_id"]["library_card_id"])
    else:
        library_card_ids.append(record["library_card_id"])

# Prepare the output file
output_file_path = 'library_card_ids.json'
random_sample_dicts = []

# Add one random record at a time until we have 30,000 records
while len(random_sample_dicts) < 30000:
    random_record = random.choice(library_card_ids)
    random_sample_dicts.append({"library_card_id": random_record})

# Save the sampled data to the output file
with open(output_file_path, 'w') as output_file:
    json.dump(random_sample_dicts, output_file, indent=4)

print(f"Exactly 30,000 random library card IDs saved to '{output_file_path}'")

Exactly 30,000 random library card IDs saved to 'library_card_ids.json'


Skopiowanie danych w ID do 30 000 rekordów wraz z randomizacją

In [36]:
# Path to the new file with uploaded data
new_file_path = 'books_data.json'

# Load the data from the file
with open(new_file_path, 'r', encoding='utf-8') as file:
    books_data = json.load(file)

# Extract only the "ID" field from each record
book_ids = []

# Add one random record at a time until we have 30,000 records
while len(book_ids) < 30000:
    random_record = random.choice(books_data)
    book_ids.append({"ID": random_record["ID"]})

# Save the sampled data to a new output file
output_file_path_books = 'books_ids.json'
with open(output_file_path_books, 'w', encoding='utf-8') as output_file:
    json.dump(book_ids, output_file, indent=4)

print(f"Exactly 30,000 random book IDs saved to '{output_file_path_books}'")

Exactly 30,000 random book IDs saved to 'books_ids.json'


Łączenie plików w jeden

In [38]:
# Paths to the uploaded files
borrowing_file_path = 'borrowing_dates.json'
large_book_ids_file_path = 'books_ids.json'
library_card_ids_file_path = 'library_card_ids.json'

# Load data from the files
with open(borrowing_file_path, 'r') as borrowing_file:
    borrowing_data = json.load(borrowing_file)

with open(large_book_ids_file_path, 'r') as book_ids_file:
    book_ids_data = json.load(book_ids_file)

with open(library_card_ids_file_path, 'r') as library_card_ids_file:
    library_card_ids_data = json.load(library_card_ids_file)

# Ensure all lists have the same length
min_length = min(len(borrowing_data), len(book_ids_data), len(library_card_ids_data))
borrowing_data = borrowing_data[:min_length]
book_ids_data = book_ids_data[:min_length]
library_card_ids_data = library_card_ids_data[:min_length]

# Combine the data into one list of dictionaries
combined_data = [
    {
        "borrowing_date": borrowing_data[i]["borrowing_date"],
        "return_date": borrowing_data[i]["return_date"],
        "book_id": book_ids_data[i]["ID"],
        "library_card_id": library_card_ids_data[i]["library_card_id"]
    }
    for i in range(min_length)
]

# Save the combined data to a new JSON file
output_combined_file_path = 'borrowing_data.json'
with open(output_combined_file_path, 'w') as combined_file:
    json.dump(combined_data, combined_file, indent=2)

output_combined_file_path

'borrowing_data.json'